In [1]:
import json
import pandas as pd
import pickle
from collections import Counter
import seaborn as sns
import matplotlib.pyplot as plt
import math
from IPython.display import display, HTML
from itertools import chain
import re
plt.rcParams['figure.dpi'] = 300

### Selecione qual base usar

In [2]:
with open("../data/heuristicas_reagrupamento/new_groups_licitacao.json") as f:
    groups_ = json.load(f)

df = pd.read_csv("../data/output/druid_fasttext/f03_items.csv.zip", sep=";") # itens licitação

with open("../data/output/druid_fasttext/results.pkl", "rb") as f:
    groups = pickle.load(f)

In [ ]:
with open("../data/heuristicas_reagrupamento/new_groups_nf.json") as f:
    groups_ = json.load(f)

df = pd.read_csv("../data/output/nota_fiscal_fasttext/f03_items.csv.zip", sep=";") # itens NF

with open("../data/output/nota_fiscal_fasttext/results.pkl", "rb") as f:
    groups = pickle.load(f)

In [3]:
# Remove os outliers e agrupa todos os itens de uma mesma família, ie, "gasolina_1", "gasolina_2", ..., "gasolina_n" => "gasolina"
inverse = {}
real_cluster = {}

for cluster, items in groups.items():
    if "_" in cluster:
        group, cod = cluster.split("_")
    else:
        cod = "-1"
        
    for item in items:
        if cod == "-1":
            inverse[item] = "outlier"
            real_cluster[item] = cluster
        else:
            inverse[item] = cluster
            real_cluster[item] = cluster

df["group"] = df.index
df.group = df.group.apply(lambda i: inverse.get(i, "outlier"))

# Salva os grupos originais para análise posterior
df["real_cluster"] = df.index
df.real_cluster = df.real_cluster.apply(lambda i: real_cluster.get(i, "outlier"))

In [4]:
unidades_medidas = df.unidades_medida.drop_duplicates().apply(lambda v:eval(v))
unidades_medidas = list(chain.from_iterable(unidades_medidas.values))
print(", ".join(unidades_medidas[:15]) + " [...]")

mg, ml, cm, mm, litro, gr, mg, ml, lt, kg, mes, ml, ano, cm, mt [...]


In [ ]:
#

# Top 20-descrições canônicas

In [5]:
def get_description_and_n_items(groups, *, max_length=70, n=20):
    group_n_itens = []
    descriptions = []
    for group in groups:
        items = df.loc[df.group == group]["original_prep"].apply(lambda lst: " ".join(eval(lst))).to_list()
        group_n_itens.append(f"{group} ({len(items)})")
        descriptions.extend(items)
        
    description_count = Counter(descriptions)
    top_n_descriptions = []
    total = sum(description_count.values())
    for description, count in description_count.most_common(n):
        top_n_descriptions.append(f"{fmt(description, max_length)} ({100 * count/total:>6.2f}%)")
        
    return group_n_itens, top_n_descriptions

def get_units_of_measure(groups):
    units_of_measure = []
    for group in groups:
        units_of_measure_by_group = df.loc[df.group == group].unidades_medida.apply(lambda v: eval(v))
        units_of_measure_by_group = list(chain.from_iterable(units_of_measure_by_group.values))
        units_of_measure.extend(units_of_measure_by_group)
            
    return list(set(units_of_measure))

def get_numbers(groups):
    numbers = []
    pattern = re.compile("^[0-9]+$")
    for group in groups:
        numbers_by_group = df.loc[df.group == group]["original_prep"].apply(lambda v: [str(e) for e in eval(v) if pattern.match(e)])
        numbers_by_group = list(chain.from_iterable(numbers_by_group.values))
        numbers.extend(numbers_by_group)
            
    return list(set(numbers))
    
def fmt(string, n):
    if len(string) > n:
        return "\"" + string[:n-3] + "..." + "\""
    else:
        return "\"" + string + "\""
    
# Is it a number or a unit of measure?
def contains_number_and_unit_of_measure(canonical_description):
    tokens = canonical_description.split()
    contains_number, contains_unit_of_measure = False, False
    pattern = re.compile("[0-9]+")
    for token in tokens:
        if pattern.match(token):
            contains_number = True
        if token in unidades_medidas:
            contains_unit_of_measure = True
            
    return contains_number and contains_unit_of_measure

def plot_hist(data: Counter, *, xlabel, filename, persist=True):
    _df = pd.DataFrame(data.items())
    sns.barplot(data=_df, x=0, y=1, color="blue")
    plt.xlabel(xlabel)
    plt.ylabel("# ocorrências")
    plt.xticks(fontsize=10)
    plt.xticks(fontsize=8, rotation=90)
    if persist:
        plt.savefig(filename, dpi=300)
    display(plt.show())

### Geral

In [6]:
   
html_str = "<table>"
html_str += "<td>Métrica</td><td>Grupos originais</td><td>Descrição canônica</td><td>Descrições frequentes</td>"
for metric in ["median", "sum"]:  
    for n in [5]:
        print(f"{metric}_{n}")
        groups = list(groups_[f"{metric}_{n}"].items())
        groups.sort(key=lambda group: len(group[1]["groups"]), reverse=True)    
        for g in groups[:10]:
            new_group = g[0]
            canonical_description = g[1]["description"]
            previous_groups = g[1]["groups"]
            previous_groups_with_n_items, frequent_descriptions = get_description_and_n_items(previous_groups)
            previous_groups_str = "<br/>".join(previous_groups_with_n_items)
            top_n_frequent_descriptions = "<ul>" + "".join(["<ol>" + x + "</ol>" for x in frequent_descriptions]) + "<ol></ol></ul>"

            html_str += "<tr>"
            html_str += f"<td>{metric}_{n}</td>"
            html_str += f"<td>{previous_groups_str}</td><td>{canonical_description}</td><td>{top_n_frequent_descriptions}</td>"
            html_str += "</tr>"
        
html_str += "</table>"
display(HTML(html_str))

median_5
sum_5


median_5,outro_0 (59)outro_1 (213)outro_2 (48)outro_3 (38)outro_5 (137)outro_6 (34)outro_7 (87)outro_8 (40)outro_9 (38)outro_10 (280)outro_11 (70)outro_12 (297),juridico outro pessoa servico terceiro,"""outro servico terceiro pessoa juridico promocao evento interesse pu..."" ( 15.88%)""outro servico terceiro pessoa juridico"" ( 7.01%)""outro servico terceiro pessoa juridico manutencao conservacao maqui..."" ( 5.44%)""outro servico terceiro pessoa juridico grafico"" ( 4.10%)""outro servico terceiro pessoa juridico apoio evento interesse publico"" ( 2.54%)""outro servico terceiro pessoa juridico locacao maquina equipamento"" ( 2.31%)""outro servico terceiro pessoa juridico manutencao conservacao bem i..."" ( 1.86%)""outro servico terceiro pessoa juridico confeccao geral"" ( 1.57%)""outro servico terceiro pessoa juridico locacao veiculo"" ( 1.27%)""outro servico terceiro pessoa juridico medico hospitalar odontologi..."" ( 0.67%)""outro servico terceiro pessoa juridico publicidade propaganda"" ( 0.60%)""outro servico terceiro pessoa juridico tecnologia informacao"" ( 0.60%)""outro servico terceiro pessoa juridico assentamento acomodacao remo..."" ( 0.60%)""outro servico terceiro pessoa juridico tecnologia informacao invest..."" ( 0.60%)""outro despesa pessoal decorrente contrato terceirizacao administrativo"" ( 0.52%)""outro"" ( 0.52%)""outro daqui mil ano"" ( 0.52%)""outro servico terceiro pessoa juridico telecomunicacao"" ( 0.45%)""outro servico pessoa juridico"" ( 0.45%)""outro servico terceiro pessoa juridico seguro geral"" ( 0.37%)"
median_5,dipirona_5 (633)dipirona_6 (396)dipirona_9 (219)dipirona_10 (526)dipirona_15 (170)dipirona_16 (159)dipirona_19 (86)dipirona_20 (238)dipirona_21 (248)dipirona_25 (566),500 dipirona mg ml sodico,"""dipirona sodico 500 mg ml"" ( 6.97%)""dipirona sodico 500 mg ml solucao oral"" ( 6.14%)""dipirona 500 mg ml"" ( 6.02%)""dipirona sodico 500 mg ml solucao injetavel"" ( 2.65%)""dipirona 500 mg ml injetavel"" ( 2.19%)""dipirona sodico 500 mg ml 2"" ( 1.76%)""dipirona sodico 500 mg ml solucao injetavel ampola 2"" ( 1.57%)""dipirona sodico 500 mg ml ampola 2"" ( 1.36%)""dipirona sodico 500 mg ml 10"" ( 1.17%)""dipirona sodico 500 mg ml injetavel ampola 2"" ( 1.14%)""dipirona 500 mg ml ampola 2"" ( 0.99%)""dipirona 500 mg ml solucao oral"" ( 0.96%)""dipirona sodico 500 mg 2 ml"" ( 0.96%)""dipirona 500 mg ml 10"" ( 0.93%)""dipirona sodico 500 mg ml solucao oral gota"" ( 0.89%)""dipirona sodico 500 mg ml solucao injetavel ver endovenoso intramus..."" ( 0.77%)""dipirona 500 mg ml 2"" ( 0.77%)""dipirona sodico 500 mg ml injetavel"" ( 0.71%)""dipirona sodico 500 mg ml solucao oral 10"" ( 0.71%)""dipirona 500 mg ml gts"" ( 0.68%)"
median_5,calcado_1 (134)calcado_4 (315)calcado_12 (64)calcado_16 (65)calcado_17 (76)calcado_18 (52)calcado_19 (113)calcado_24 (47)calcado_26 (53),botina calcado ocupacional seguranca tipo,"""calcado seguranca tipo botina cor preto n 39"" ( 1.09%)""calcado seguranca tipo botina cor preto n 43"" ( 0.98%)""calcado seguranca tipo botina cor preto n 40"" ( 0.98%)""calcado seguranca tipo botina cor preto n 42"" ( 0.98%)""calcado seguranca tipo botina cor preto n 41"" ( 0.87%)""calcado seguranca protecao pe perna contra umidade proveniente oper..."" ( 0.76%)""calcado seguranca tipo botina cor preto n 44"" ( 0.76%)""calcado seguranca tipo botina n 38"" ( 0.65%)""calcado seguranca uso profissional tipo bota impermeavel preto sem ..."" ( 0.65%)""calcado seguranca com biqueiro aco uso profissional tipo botina mod..."" ( 0.65%)""calcado para trabalho cozinha ocupacional tipo sapato fechamento el..."" ( 0.65%)""calcado seguranca modelo bota cano medio cabedal pvc cor branco ent..."" ( 0.65%)""calcado ocupar tipo bota n 38"" ( 0.54%)""calcado seguranca tipo botina 42"" ( 0.54%)""calcado seguranca tamanho 38 tipo sapato feminino modelo blatt resi..."" ( 0.44%)""calcado tipo sapato mod blatt cor preto n 38"" ( 0.44%)""calcado seguranca tipo botina nobuck n 38"" ( 0.44%)""calcado seguranca sem biq

### Contém número ou unidade de medida

In [7]:
html_str = "<table>"
html_str += "<td>Métrica</td><td>Grupos originais</td><td>Descrição canônica</td><td>Descrições frequentes</td>"
for metric in ["median", "sum"]:  
    n = 5
    print(metric)
    groups = list(groups_[f"{metric}_{n}"].items())
    groups.sort(key=lambda group: len(group[1]["groups"]), reverse=True)
    
    selected_groups_count = 0
    for g in groups:
        new_group = g[0]
        canonical_description = g[1]["description"]
                
        if not contains_number_and_unit_of_measure(canonical_description):
            continue
            
        if selected_groups_count > 20:
            break
            
        previous_groups = g[1]["groups"]
        previous_groups_with_n_items, frequent_descriptions = get_description_and_n_items(previous_groups)
        previous_groups_str = "<br/>".join(previous_groups_with_n_items)
        top_n_frequent_descriptions = "<ul>" + "".join(["<ol>" + x + "</ol>" for x in frequent_descriptions]) + "<ol></ol></ul>"

            
        selected_groups_count += 1
        html_str += "<tr>"
        html_str += f"<td>{metric}_{n}</td>"
        html_str += f"<td>{previous_groups_str}</td><td>{canonical_description}</td><td>{top_n_frequent_descriptions}</td>"
        html_str += "</tr>"
        
html_str += "</table>"
display(HTML(html_str))

median
sum


median_5,dipirona_5 (633)dipirona_6 (396)dipirona_9 (219)dipirona_10 (526)dipirona_15 (170)dipirona_16 (159)dipirona_19 (86)dipirona_20 (238)dipirona_21 (248)dipirona_25 (566),500 dipirona mg ml sodico,"""dipirona sodico 500 mg ml"" ( 6.97%)""dipirona sodico 500 mg ml solucao oral"" ( 6.14%)""dipirona 500 mg ml"" ( 6.02%)""dipirona sodico 500 mg ml solucao injetavel"" ( 2.65%)""dipirona 500 mg ml injetavel"" ( 2.19%)""dipirona sodico 500 mg ml 2"" ( 1.76%)""dipirona sodico 500 mg ml solucao injetavel ampola 2"" ( 1.57%)""dipirona sodico 500 mg ml ampola 2"" ( 1.36%)""dipirona sodico 500 mg ml 10"" ( 1.17%)""dipirona sodico 500 mg ml injetavel ampola 2"" ( 1.14%)""dipirona 500 mg ml ampola 2"" ( 0.99%)""dipirona 500 mg ml solucao oral"" ( 0.96%)""dipirona sodico 500 mg 2 ml"" ( 0.96%)""dipirona 500 mg ml 10"" ( 0.93%)""dipirona sodico 500 mg ml solucao oral gota"" ( 0.89%)""dipirona sodico 500 mg ml solucao injetavel ver endovenoso intramus..."" ( 0.77%)""dipirona 500 mg ml 2"" ( 0.77%)""dipirona sodico 500 mg ml injetavel"" ( 0.71%)""dipirona sodico 500 mg ml solucao oral 10"" ( 0.71%)""dipirona 500 mg ml gts"" ( 0.68%)"
median_5,clonazepam_0 (268)clonazepam_1 (175)clonazepam_3 (227)clonazepam_6 (296)clonazepam_7 (183)clonazepam_8 (51)clonazepam_9 (166)clonazepam_19 (435),2 5 clonazepam mg ml,"""clonazepam 2 5 mg ml"" ( 20.32%)""clonazepam 2 5 mg ml solucao oral"" ( 13.27%)""clonazepam 2 5 mg ml gota"" ( 6.83%)""clonazepam gota 2 5 mg ml"" ( 3.55%)""clonazepam 2 5 mg ml solucao oral frasco 20"" ( 3.28%)""clonazepam 2 5 mg ml frasco 20"" ( 2.67%)""clonazepam 2 5 mg ml sol oral"" ( 2.50%)""clonazepam solucao oral 2 5 mg ml"" ( 2.39%)""clonazepam 2 5 mg ml gota 20"" ( 2.28%)""clonazepam 2 5 mg ml 20"" ( 2.22%)""clonazepam 2 5 mg ml solucao"" ( 1.83%)""clonazepam 2 5 mg gota"" ( 1.39%)""clonazepam 2 5 mg ml 20 gota"" ( 1.33%)""clonazepam 2 5 mg ml frasco com 20"" ( 1.33%)""clonazepam 2 5 mg ml gts"" ( 1.11%)""clonazepam 2 5 mg ml solucao oral frasco com 20"" ( 1.00%)""clonazepam 2 5 mg ml solucao oral 20"" ( 0.94%)""clonazepam 2 5 mg ml gota frasco com 20"" ( 0.83%)""clonazepam 2 5 mg ml gota frasco"" ( 0.78%)""clonazepam 2 5 mg ml solucao oral frasco gotejador 20"" ( 0.72%)"
median_5,miconazol_0 (70)miconazol_6 (145)miconazol_7 (50)miconazol_13 (97)miconazol_14 (36)miconazol_16 (72)miconazol_23 (37)miconazol_24 (430),20 g mg miconazol nitrato,"""miconazol nitrato 20 mg g creme vaginal"" ( 23.69%)""miconazol nitrato 20 mg g creme"" ( 14.62%)""miconazol nitrato 20 mg g"" ( 8.75%)""miconazol nitrato 20 mg g locao"" ( 6.30%)""miconazol nitrato 20 mg g creme dermatologico"" ( 3.09%)""miconazol nitrato"" ( 2.88%)""miconazol nitrato 20 mg creme vaginal"" ( 2.03%)""miconazol nitrato 20 mg g creme vaginal bisnaga 80 aplicador"" ( 1.81%)""miconazol nitrato 20 mg g creme dermatologico bisnaga 28"" ( 1.81%)""miconazol nitrato 20 mg g creme vaginal 80"" ( 1.49%)""miconazol nitrato 20 mg g creme vaginal bisnaga com 80 aplicador"" ( 1.07%)""miconazol nitrato 20 mg g 80"" ( 0.96%)""miconazol nitrato 20 mg g creme vaginal tubo com 80"" ( 0.75%)""miconazol nitrato 20 mg g crer"" ( 0.64%)""miconazol nitrato 20 mg creme bisnaga 28 grs"" ( 0.64%)""miconazol nitrato 20 mg g creme locao frasco 30 grama"" ( 0.64%)""miconazol nitrato 20 mg g creme vaginal frasco contender 80 grama"" ( 0.64%)""miconazol nitrato 20 mgg creme vaginal"" ( 0.64%)""miconazol nitrato 20 mg g creme vaginal bisnaga 80"" ( 0.64%)""miconazol nitrato 20 mg g creme dermatologico tubo 28 grama"" ( 0.64%)"
median_5,sulfadiazina_5 (147)sulfadiazina_6 (173)sulfadiazina_14 (121)sulfadiazina_15 (81)sulfadiazina_21 (70)sulfadiazina_22 (148)sulfadiazina_30 (36),10 g mg prata sulfadiazina,"""sulfadiazina prata 10 mg g creme"" ( 13.27%)""sulfadiazina prata 10 mg g"" ( 12.50%)""sulfadiazina prata 10 mg"" ( 3.61%)""sulfadiazina prata 10 mg g creme dermatologico"" ( 2.32%)""sulfadiazina prata 10 mg g pote 400"" ( 1.55%)""sulfadiazina prata 10 mg g creme dermatologico pote com 400"" ( 1.55%)""sulf

### Distribuição números e unidades de medidas

In [ ]:
html_str = "<table>"
html_str += "<td>Métrica</td><td>Novo Grupo</td><td>Grupos originais</td><td>Un. de medidas únicas</td><td>Números unicos</td>"
for metric in ["median", "sum"]:  
    n = 5
    print(metric)
    groups = list(groups_[f"{metric}_{n}"].items())
    groups.sort(key=lambda group: len(group[1]["groups"]), reverse=True)
    groups = list(filter(lambda group: len(group[1]["groups"]) > 1, groups))
    
    hist_unique_units_of_measure, hist_unique_numbers = Counter(), Counter()
    selected_groups_count = 0
    for g in groups:
        new_group = g[0]
        canonical_description = g[1]["description"]
                
        if not contains_number_and_unit_of_measure(canonical_description):
            continue
            
        previous_groups = g[1]["groups"]
        previous_groups_with_n_items, frequent_descriptions = get_description_and_n_items(previous_groups)
        previous_groups_str = ", ".join(previous_groups_with_n_items)
        
        unique_units_of_measure = get_units_of_measure(previous_groups)
        hist_unique_units_of_measure[len(unique_units_of_measure)] += 1
        unique_units_of_measure_str = ", ".join(unique_units_of_measure)
        
        unique_numbers = get_numbers(previous_groups)
        hist_unique_numbers[len(unique_numbers)] += 1
        unique_numbers_str = ", ".join(unique_numbers)
        
        if selected_groups_count > 20:
            continue
            
        selected_groups_count += 1
        html_str += "<tr>"
        html_str += f"<td>{metric}_{n}</td>"
        html_str += f"<td>{new_group}</td><td>{previous_groups_str}</td><td>{unique_units_of_measure_str}</td><td>{unique_numbers_str}</td>"
        html_str += "</tr>"
    
    print(metric, n)
    plot_hist(hist_unique_units_of_measure, xlabel="unidades de medidas únicas", filename=f"avaliacao_reagrupamento_unidade_medida_{metric}_5.jpg")
    plot_hist(hist_unique_numbers, xlabel="números únicos", filename=f"avaliacao_reagrupamento_numeros_{metric}_5.jpg")
        
html_str += "</table>"
display(HTML(html_str))

# Histograma dos tamanho dos grupos após o reagrupamento

In [ ]:
for metric in groups_:
    counter = Counter()
    for key in groups_[metric]:
        n_regrouped = key.count("_") 
        if n_regrouped < 2:
            continue
        counter[n_regrouped] += 1
    if len(counter) == 0:
        print("ups!")
        continue
    _df = pd.DataFrame.from_dict(counter, orient='index').reset_index()
    sns.barplot(data=_df, x="index", y=0, color="blue")
    plt.xlabel("tamanho dos grupos")
    plt.ylabel("quantidade de grupos")
    print(metric)
    display(plt.show())

# Histograma da quantidade de itens em cada grupo após o reagrupamento

In [ ]:
step = 30
for metric in groups_:
    counter = Counter()
    for key in groups_[metric]:
        n_regrouped = key.count("_") 
        if n_regrouped < 2:
            continue
            
        original_groups = groups_[metric][key]["groups"]
        num_itens = len(df.loc[df.group.isin(original_groups)])
        counter[num_itens] += 1
    
    buckets = {}
    for k, v in counter.most_common():
        bucket = k//step if k < 1000 else 1000
        if bucket not in buckets:
            if bucket == 1000:
                buckets[bucket] = {"bucket": bucket, "# de itens": f"(1.000, +∞]", "# grupos": 0}
            else:
                buckets[bucket] = {"bucket": bucket, "# de itens": f"({bucket * step}, {(bucket + 1) * step}]", "# grupos": 0}
        buckets[bucket]["# grupos"] += v
    if len(buckets) == 0:
        print("ups!")
        continue
        
    _df = pd.DataFrame.from_records(list(buckets.values())).reset_index().sort_values(by="bucket")  
    sns.barplot(data=_df, x="# de itens", y="# grupos", color="blue")
    plt.xticks(rotation=90)
    print(f"{metric}")
    display(plt.show())

In [ ]:
d = {"a": 10, "b": 20, "c": 30}
__df = pd.DataFrame(d.items())
sns.barplot(data=__df, x=0, y=1, color="blue")
plt.xlabel("Foo")
plt.ylabel("Bar")
plt.xticks(fontsize=10)
plt.xticks(fontsize=8, rotation=90)
display(plt.show())

In [ ]:
df